In [132]:
class AssociationAnalysis:
    
    def __init__(self, D, verbose=True, lemmatization=True, min_support=3/10, min_confidence=6/10, autostart=True):
        '''
            D is the data basis containing all transaction sets.
            One transaction for example contains the set of items you bought (e.g. `{'strawberries', 'blueberries'}`).
            The cardinality of D is the amount of transactions in the data basis D.
            D contains items from the statistical population of items P (all possible items).
            
            X is any subset of the population P (k-item-set with k>=1) or a string which will be automatically converted to a 1-element subset (e.g. X='lemon juice' will be converted to X={'lemon juice'}).
            
            Lemmatization is enabled by default and requires downloaded lemmatization packages from nltk:
            If you haven't done so, please execute:
                `
                import nltk
                nltk.download()
                `
            By default (`autostart=True`), the association analysis will be carried out immediately. 
            If you set autostart to False, you can manually start the analysis by calling the `AssociationAnalysis.apriori()` method.
        
            Supports, confidences and lifts are memoized once calculated.
        '''
        # remove unnecessary object variables (incl. P_...) and instead pass as parameter 
        # fix lemmatization for sets (maybe by converting unhashable sets to frozensets then back to sets)
        
        self.verbose = verbose
        self.lemmatization = lemmatization
        self.min_support = min_support
        self.min_confidence = min_confidence
        
        if self.lemmatization:
            import nltk
            # nltk.download() required
            from nltk.stem import WordNetLemmatizer 
            self.lemmatizer = WordNetLemmatizer()
            self.D = self.lemmatize_list_of_sets(D)
        else:
            self.D = D
        
        self.D_cardinality = len(D)

        self.P = set().union(*D)
        self.P_cardinality = len(self.P)
        self.P_remaining_items = []
        self.P_remaining_items = list(self.P)
        self.P_remaining_subsets = {}
        self.P_remaining_subsets[1] = self.P
        self.supports = {} # key is X
        self.confidences = {} # key is tuple of two elements (X, Y)
        self.lifts = {} # key is tuple of two elements (X, Y)
        
        if autostart: self.apriori()
    
    def n(self, X, prepare=False):
        '''
            Absolute prevalence of X in data basis D
            Requirement: |X| >= 1
        '''
        if prepare:
            X = self.prepare_subset(X)
        if not isinstance(X, set):
            raise ValueError(f'Given parameter X is not a set. X={X}')
        n = 0
        for transaction in self.D:
            if X.issubset(transaction): 
                if X: n += 1
        # if self.verbose: self.latex(rf'$n(${X}$)={n}$')
        return n
    
    def support(self, *X, prepare=False):
        '''
            support(X_1, X_2, ..., X_n) = support(*X) for a given interable X.
            The support is the relative prevalence of item(-s) in data basis D.
            It shows, in which percentage of transactions the given item set was a subset of the transaction set.
            If `prepare=True`, prepare the parameter `*X` via `self.prepare_subset()`.
        '''
        if prepare:
            X = self.prepare_subset(X)
        else:
            X = self.X_to_subset(X)
        
        memoize_dict = self.supports
        memoize_key = self.set_or_tuple_to_hashable_set(X)
        memoize_read = self.memoize_read(memoize_dict, memoize_key)
        if memoize_read:
            return memoize_read
        
        else:  
            n_X = self.n(X,prepare=prepare)

            support = n_X/self.D_cardinality

            self.memoize_save(memoize_dict, memoize_key, support)

            output_string = rf'$support(X = $ {X}$)$ $= \frac{{n(X)}}{{|D|}} = \frac{{{n_X}}}{{{self.D_cardinality}}}$'
            self.color_output_if(output_string=output_string, condition=(support < self.min_support))
            return support

    def prepare_X_and_Y_from_Z(self, *Z, prepare=False):
        '''
            Input: *Z as parameter list of items
            Output: X, Y
            Split the list of parameters into `X` and `Y`.
            `Y` is the one-element set containing only the last parameter passed in `*Z`
            `X` is the set of all other parameters in `*Z` except for `Y`.
            Cardinality of `y` is exactly 1.
            Minimum cardinality of `Z` is 2.
        '''
        if prepare:
            Y = self.prepare_subset(Z[-1])
            X = self.prepare_subset(Z[:-1])
        else:
            Y = self.scalar_to_set(Z[-1]) # only last element
            X = self.tuple_to_set(Z[:-1]) # every element but the last          
        return X, Y

    def memoize_read(self, dictionary, key):
        if key in dictionary:
            return dictionary[key] 
        else:
            return None

    def memoize_save(self, dictionary, key, value):
        dictionary[key] = value

    def memoize_X_Y_key(self, X, Y):
        return (self.set_or_tuple_to_hashable_set(X), self.set_or_tuple_to_hashable_set(Y))

    def confidence(self, *Z, prepare=False):
        '''
            $confidence(*Z) = confidence(*X, *Y)$
            $ = confidence(X_1, X_2, ..., X_n, Y) = confidence(X \to Y) = \frac{n(X \cup Y)}{n(X)}$
            
            If `prepare=True`, prepare the parameters `*X` and `y` via `self.prepare_subset()`.
        '''
        X, Y = self.prepare_X_and_Y_from_Z(*Z)
        
        memoize_dict = self.confidences
        memoize_key = self.memoize_X_Y_key(X, Y)
        memoize_read = self.memoize_read(memoize_dict, memoize_key)
        
        if memoize_read:
            return memoize_read
        
        else:
            n_X = self.n(X,prepare=prepare)

            X_union_Y = set().union(X, Y)
            n_X_union_Y = self.n(X_union_Y)
            if n_X == 0:
                confidence = None
            else:
                confidence = n_X_union_Y/n_X

            self.memoize_save(memoize_dict, memoize_key, confidence)

            output_string = rf'$confidence(X= $ {X} $ \to Y=$ {Y}$)$ $ = \frac{{n(X \cup Y)}}{{n(X)}} = \frac{{{n_X_union_Y}}}{{{n_X}}}$'
            self.color_output_if(output_string=output_string, condition=(confidence < self.min_confidence))

            return confidence
    
    def lift(self, *Z, prepare=False):
        '''
        $lift(*Z) = lift(*X, *Y) = lift(X_1, X_2, ..., X_n, Y) = lift(X \to Y) = \frac{confidence(X \to y)}{support(Y)}$
        '''
        X, Y = self.prepare_X_and_Y_from_Z(*Z)
        confidence = self.confidence(*Z)
        support = self.support(*Y)
        
        memoize_dict = self.lifts
        memoize_key = self.memoize_X_Y_key(X, Y)
        memoize_read = self.memoize_read(memoize_dict, memoize_key)
        
        if memoize_read:
            return memoize_read
        else:
            lift = confidence / support
            
            self.memoize_save(memoize_dict, memoize_key, lift)
            
            if self.verbose:
                output_string = rf'$lift(X= $ {X} $ \to Y=$ {Y}$)$ $ = \frac{{confidence(X \to Y)}}{{support(Y)}} = \frac{{{confidence}}}{{{support}}}$'
                self.color_output_if(output_string=output_string, condition=(lift < 1))
            return lift
        

    def filter_P_by_min_support(self, k):
        remove_sets = [X for X, support_value in self.supports.items() if support_value < self.min_support]
                
        P_snapshot_k = self.P_remaining_subsets[k]
        self.P_remaining_subsets[k] = []
        for subset in P_snapshot_k:
            if subset not in remove_sets:
                self.P_remaining_subsets[k].append(subset)
        self.P_remaining_items = frozenset().union(*self.P_remaining_subsets[k])
    
    def apriori(self):
        frequent_subsets = self.apriori_step_1()
        self.apriori_step_2(frequent_subsets)

    def set_or_tuple_to_hashable_set(self, X):
        '''
            Conversion of X to a hashable frozenset.
            Input: A given tuple or set X 
            Output: hashable frozenset
        '''
        if isinstance(X, tuple):
            generate_frozenset_from = self.tuple_to_set(X)
        else: # e.g. if isinstance(X, set)
            generate_frozenset_from = X
        return frozenset(generate_frozenset_from)
        
    def apriori_step_1(self):
        '''
            Calculate frequent subsets $F$ with support >= min_support.
        '''
        import itertools
        
        if self.verbose: self.latex("### Supports")
        k = 1
        while(k <= len(self.P) and self.P_remaining_items):
            if self.verbose: self.latex(f"$k = {k}$")
            
            self.P_remaining_subsets[k] = [frozenset(X) for X in itertools.combinations(list(self.P_remaining_items), k)]
            for X in self.P_remaining_subsets[k]:
                X_hashable_set = self.set_or_tuple_to_hashable_set(X)
                self.supports[X_hashable_set] = self.support(*X)
            self.filter_P_by_min_support(k)
            k += 1
        frequent_subsets = [set([*X]) for X, support_value in self.supports.items() if support_value >= self.min_support]
        if self.verbose:
            frequent_subsets_output = "### Frequent subsets $F$"
            frequent_subsets_output += '\n' + ', \n'.join(str(subset) for subset in frequent_subsets)
            self.latex(frequent_subsets_output)
        return frequent_subsets
    
    def apriori_step_2(self, frequent_subsets):
        '''
            Calculate rules of form $confidence(X \to Y) \land |Y| = 1 
            \land confidence >= min_{confidence}$ for frequent subsets $F$.
        '''
        for Z in frequent_subsets:
            Z_ordered = list(Z)
            k = 1
            H = {}
            # put every element in the frequent subset X last once to calculate all confidence rules
            while k < len(Z_ordered)-1:
                # current Y at position k
                Z_without_current_Y = Z_ordered[:k] + Z_ordered[(k + 1):]
                X = Z_without_current_Y
                if Z_ordered[k]:
                    Y = Z_ordered[k]
                    confidence_current_Y = self.confidence(*X, Y)
                    
                    if (confidence_current_Y > self.min_confidence):
                        lift_k = self.lift(*Z_without_current_Y)
                        
                k = k + 1
            
    def tuple_to_set(self, X):
        '''
            Helper function to convert a given n-tuple to an n-element set.
        '''
        if isinstance(X, tuple):
            X = set(X)
        return X
    
    def scalar_to_set(self, X):
        '''
            Helper function to convert a given string or int scalar value to a 1-element set
        '''
        if isinstance(X, str) or isinstance(X, int):
            X = set([X])
        return X
    
    def X_to_subset(self, X):
        '''
            If X is not a subset, but an iterable (e.g. tuple), 
            convert the given k-element iterable to a subset including all k-elements.
        '''
        X = self.tuple_to_set(X)        
        X = self.scalar_to_set(X)
        return X
    
    def prepare_subset(self, X, lemmatization=None):
        '''
            Helper function to prepare a subset X.
            If a tuple is given, convert the n-element tuple to an n-element set.
            If a string is given, convert the string to a 1-element set.
            If lemmatization is enabled, lemmatize the set.
        '''
        if lemmatization is None:
            lemmatization = self.lemmatization   
            
        X = self.X_to_subset(X)
        
        if lemmatization:
            X = self.lemmatize_set(X)
        return X

    def lemmatize_list_of_sets(self, D):
        return [self.lemmatize_set(transaction) for transaction in D]
    def lemmatize_set(self, set_of_items):
        import collections
        if(isinstance(set_of_items, collections.Hashable)):
            return {self.l(item) for item in set_of_items}
        else:
            return set_of_items
    def l(self, item):
        '''
         Helper function l(item) = lemmatize(item) returns the lemmatized version of a string item.
        '''
        import collections
        if(isinstance(item, collections.Hashable)):
            l = self.lemmatizer.lemmatize(item)
        else:
            l = item
        return l

    def latex(self, string):
        '''
        Helper function to output latex in verbose mode.
        '''
        if self.verbose:
            from IPython.display import display, Markdown
            display(Markdown(rf"""{string}"""))
            
    def color_output_if(self, output_string, condition, condition_color='#ccc', default_color='#000'):
        '''
            If verbose=True:
                If `condition`, display `output_string` in `condition_color` (by default the lighter color HEX #ccc).
                Else, display `output_string` in the default color black (HEX #000).
        '''
        if self.verbose:
            if condition:
                output_string = rf'<font color="{condition_color}">{output_string}</font>'    
            self.latex(output_string)

In [133]:
D = []
D.append({'strawberries','lemon juice'})
D.append({'strawberries', 'toothbrush', 'oatmilk', 'lentils', 'lemon juice'})
D.append({'lemon juice', 'toothbrush', 'oatmilk'})
D.append({'strawberries', 'lemon juice', 'toothbrush'})
D.append({'strawberries', 'oatmilk'})

association_analysis = AssociationAnalysis(D,lemmatization=True)
# association_analysis.support('strawberries', 'lemon juice',prepare=True)
# association_analysis.confidence('strawberries', 'lemon juice', 'toothbrush', prepare=True)

### Supports

$k = 1$

$support(X = $ {'toothbrush'}$)$ $= \frac{n(X)}{|D|} = \frac{3}{5}$

$support(X = $ {'oatmilk'}$)$ $= \frac{n(X)}{|D|} = \frac{3}{5}$

$support(X = $ {'lemon juice'}$)$ $= \frac{n(X)}{|D|} = \frac{4}{5}$

...

$confidence(X= $ {'toothbrush'} $ \to Y=$ {'lemon juice'}$)$ $ = \frac{n(X \cup Y)}{n(X)} = \frac{3}{3}$

$lift(X= $ {'toothbrush'} $ \to Y=$ {'lemon juice'}$)$ $ = \frac{confidence(X \to Y)}{support(Y)} = \frac{1.0}{0.8}$

$confidence(X= $ {'toothbrush', 'strawberries'} $ \to Y=$ {'lemon juice'}$)$ $ = \frac{n(X \cup Y)}{n(X)} = \frac{2}{2}$

$confidence(X= $ {'toothbrush'} $ \to Y=$ {'strawberries'}$)$ $ = \frac{n(X \cup Y)}{n(X)} = \frac{2}{3}$

<font color="#ccc">$lift(X= $ {'toothbrush'} $ \to Y=$ {'strawberries'}$)$ $ = \frac{confidence(X \to Y)}{support(Y)} = \frac{0.6666666666666666}{0.8}$</font>